
Reconhecimento de comandos de voz

image-2.png

Base de dados: Mini Speech Commands

O conjunto de dados original consiste em mais de 105.000 arquivos de áudio no formato de arquivo de áudio WAV (Waveform) de pessoas dizendo 35 palavras diferentes. Mais detalhes sobre a base de dados podem ser vistos nestes link: speech_commands

Para economizar tempo com o carregamento de dados, será usada uma versão menor do conjunto de dados de Comandos de Fala chamada mini_speech_commands que contém clipes de áudio curtos (um segundo ou menos) de 8 comandos: "down", "go", "left", "no", "right", "stop", "up" e "yes".

A taxa de amostragem para este conjunto de dados é de 16kHz.

In [26]:
#Importando as bibliotecas

# Instala a versão específica 0.8.1 da biblioteca librosa.
!pip install --upgrade librosa
#==0.8.1
!pip install --upgrade numpy

# Verifica a versão do librosa
#print(librosa.__version__)
#librosa.__version__

  Using cached librosa-0.10.1-py3-none-any.whl.metadata (8.3 kB)
Using cached librosa-0.10.1-py3-none-any.whl (253 kB)
  Attempting uninstall: librosa
    Found existing installation: librosa 0.8.1
    Uninstalling librosa-0.8.1:
      Successfully uninstalled librosa-0.8.1


In [27]:

# Importação de bibliotecas e módulos necessários.
import glob
import os
import pathlib
import random
from datetime import datetime
import numpy as np
import librosa
import librosa.display as ld
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Audio
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models
import seaborn as sns
sns.set()
from tqdm import tqdm


In [6]:
import os

dir_dataset = "D:/Dados/material_complementar_reconhecimento_voz/mini_speech_commands/"

if os.path.exists(dir_dataset):
    print("O diretório existe.")
else:
    print("O diretório não existe.")

O diretório existe.


In [7]:
#Análise exploratória de dados (EDA)

#Baixando e carregando o dataset Mini Speech Commands
# Baixa e descompacta o dataset Mini Speech Commands.
#!wget http://storage.googleapis.com/download.tensorflow.org/data/mini_speech_commands.zip -O mini_speech_commands.zip
#!unzip mini_speech_commands.zip -d dir_files
#!rm mini_speech_commands.zip


##Comando para Vscode
import os
import urllib.request
import zipfile

# URL do arquivo zip
url = "http://storage.googleapis.com/download.tensorflow.org/data/mini_speech_commands.zip"

# Nome do arquivo zip
zip_filename = "mini_speech_commands.zip"

# Diretório de destino para extrair o arquivo zip
extract_dir = "D:/Dados/material_complementar_reconhecimento_voz/mini_speech_commands/"

# Baixar o arquivo zip
urllib.request.urlretrieve(url, zip_filename)

# Extrair o arquivo zip
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# Remover o arquivo zip após a extração
os.remove(zip_filename)

In [37]:
# Cria uma lista dos comandos disponíveis no dataset.
#Bibliotecas auxiliares
import os
import glob

#Google Colab
#commands = []
#dir_comands = "D:/Dados/Material_complementar_reconhecimento_voz/mini_speech_commands/mini_speech_commands/"

#for name in glob.glob(str(dir_comands) + '/*' + os.path.sep):
#  print(name.split('/')[-2])
#  commands.append(name.split('/')[-2])


# VSCode Jupyter
commands = []
dir_commands = "D:/Dados/Material_complementar_reconhecimento_voz/mini_speech_commands/mini_speech_commands"

for name in glob.glob(os.path.join(dir_commands, '*')):
    print(os.path.basename(name))
    commands.append(os.path.basename(name))

down
go
left
no
README.md
right
stop
up
yes
